# Ujjwal Kishor Sahoo (21293)
# Question 1 (Code Mixed Text Classification)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [69]:
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/train.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/val.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/humor/train.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/humor/val.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/sarcasm/train.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/sarcasm/val.csv

--2025-03-18 16:42:42--  https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/sarcasm/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 472095 (461K) [text/plain]
Saving to: ‘train.csv.2’

train.csv.2         100%[===================>] 461.03K  --.-KB/s    in 0.008s  

2025-03-18 16:42:42 (59.1 MB/s) - ‘train.csv.2’ saved [472095/472095]

--2025-03-18 16:42:42--  https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/sarcasm/val.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5

In [89]:
import pandas as pd

#Loading the dataset
file_path= "/content/train.csv"

df= pd.read_csv(file_path)

df.head()


,Sentence,Tag
0,#hariyana mey ek week mey teen bachchiyo ke Sa...,1
1,indira Gandhi ko marne wala sikh.Rajiv Gandhi ...,1
2,ishliye corruption ke jariye sab ki khoon choo...,1
3,Pakistaniyon ko aisi news Maamul sa ho Gaya ha...,0
4,Apne national anthem ko change karo and yeh li...,1


In [26]:
missing_values = df.isnull().sum()
print("\nMissing Values per Column:")
print(missing_values)


Missing Values per Column:
Sentence    0
Tag         0
dtype: int64


In [27]:

sentences = df["Sentence"].tolist()
tags = df["Tag"].tolist()

#Display first few sentences
for i in range(5):
    print(f"Sentence: {sentences[i]}")
    print(f"Tag: {tags[i]}")
    print("-" * 50)

Sentence: #hariyana mey ek week mey teen bachchiyo ke Saath gang rape hue par hamari aatma ko ye Khabar jhakjhorti nahi kyunki paththar ho Gaye Hai yahaa Insaan .sabke sab gunaahgaar ..betiyo ko is tarah bachaoge ? @PMOIndia
Tag: 1
--------------------------------------------------
Sentence: indira Gandhi ko marne wala sikh.Rajiv Gandhi ko marne wali tamil ki thi,to Terrorism ko paida karne waly palny wala khud india haI.mind it https://twitter.com/chintskap/status/875447907922567170 …
Tag: 1
--------------------------------------------------
Sentence: ishliye corruption ke jariye sab ki khoon choos rahe heine
Tag: 1
--------------------------------------------------
Sentence: Pakistaniyon ko aisi news Maamul sa ho Gaya hai  jaise udar Roz Bomb blasts hote hai..
Tag: 0
--------------------------------------------------
Sentence: Apne national anthem ko change karo and yeh line add karo “chutiyeh zammen shaban, terristo Ka yeh Jahan” terrorism Itna phela hai ke koi company porkistan mei

## Words level N grams

In [28]:
vectorizer= CountVectorizer(ngram_range=(1,3))  #Preprocessing
X_train= vectorizer.fit_transform(sentences)
Y_train= tags

#Display first few features
print("Sample Features (First 10):")
print(vectorizer.get_feature_names_out()[:10])


Sample Features (First 10):
['000' '000 bandi' '000 bandi rape' '000 rape' '000 rape huehoon' '01'
 '01 27' '01 27 pakistani' '01karn' '01karn status']


### Multinomial Naive Bayes

In [29]:
model= MultinomialNB()
model.fit(X_train, Y_train)

MultinomialNB()

In [30]:
df_val= pd.read_csv('/content/val.csv')

df_val.head()

,Sentence,Tag
0,Sahee bolay ho kal Pakistani ka rape hoga,0
1,rape. Pyaar rape se kum nhi hai,0
2,ye log mandir me hi q jakar rape krte kaya bi ...,0
3,Delhi-Dehradun train mein 1seat par baithne k ...,0
4,#GadhaAkallesh ne Kairana/Sahibabad/Buxar se H...,1


In [31]:
#Extracting sentences from validtion set
val_sentences= df_val["Sentence"].tolist()
val_labels= df_val["Tag"].tolist()


### Char level N-Grams

In [32]:

vectorizer1= CountVectorizer(analyzer='char', ngram_range=(1,4))

X_all= vectorizer1.fit_transform(sentences + val_sentences)
X_train= vectorizer1.transform(sentences)
X_val= vectorizer1.transform(val_sentences)

model.fit(X_train, Y_train)

#Evaluation
y_pred = model.predict(X_val)

print("Model Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))


Model Accuracy: 0.6630196936542669

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.76      0.75       309
           1       0.48      0.46      0.47       148

    accuracy                           0.66       457
   macro avg       0.61      0.61      0.61       457
weighted avg       0.66      0.66      0.66       457



### To make similar number of counts

In [33]:
from imblearn.over_sampling import RandomOverSampler

oversample= RandomOverSampler(random_state=42)

X_resampled, Y_resampled= oversample.fit_resample(X_train, Y_train)

In [34]:
pd.Series(Y_resampled).value_counts()

,count
1,2307
0,2307


In [35]:
model.fit(X_resampled, Y_resampled)

y_pred = model.predict(X_val)

print("Model Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))


Model Accuracy: 0.6148796498905909

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.64      0.69       309
           1       0.43      0.56      0.49       148

    accuracy                           0.61       457
   macro avg       0.59      0.60      0.59       457
weighted avg       0.65      0.61      0.63       457



### SVM

### Improving accuracy by neglecting stop words etc

In [36]:
import re

#Preprocessing
def preprocess_text(text):
    text= text.lower()  #lowercase
    text= re.sub(r'\d+', '', text)  #Remove numbers
    text= re.sub(r'[^\w\s]', '', text)  #Remove punctuation
    return text


sentences2= [preprocess_text(sentence) for sentence in sentences]
val_sentences2= [preprocess_text(sentence) for sentence in val_sentences]

vectorizer_new= CountVectorizer(ngram_range=(1,3), stop_words='english')  #Removes common words
X_train1= vectorizer_new.fit_transform(sentences2)
X_val1= vectorizer_new.transform(val_sentences2)


In [37]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
svm_model= SVC(kernel='linear', class_weight='balanced')

In [38]:
svm_model.fit(X_train1, Y_train)

#Evaluation
y_pred= svm_model.predict(X_val1)

print("Model Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))

Model Accuracy: 0.700218818380744

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.84      0.79       309
           1       0.55      0.41      0.47       148

    accuracy                           0.70       457
   macro avg       0.65      0.63      0.63       457
weighted avg       0.68      0.70      0.69       457



## Logistic Regression

In [39]:
logistic_model= LogisticRegression(max_iter=500)
logistic_model.fit(X_resampled, Y_resampled)

#Evaluation
y_pred= logistic_model.predict(X_val)

print("Logistic Regression Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))

Logistic Regression Accuracy: 0.6630196936542669

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.75      0.75       309
           1       0.48      0.47      0.48       148

    accuracy                           0.66       457
   macro avg       0.61      0.61      0.61       457
weighted avg       0.66      0.66      0.66       457



## Random Forest

In [40]:
from sklearn.ensemble import RandomForestClassifier
rf_model= RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_resampled, Y_resampled)

RandomForestClassifier(random_state=42)

In [41]:
y_pred= rf_model.predict(X_val)
print("Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))

Accuracy: 0.737417943107221

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.89      0.82       309
           1       0.64      0.43      0.51       148

    accuracy                           0.74       457
   macro avg       0.70      0.66      0.67       457
weighted avg       0.72      0.74      0.72       457



## UnderSampling

# Best Model which is giving the F1 score as 56%

In [42]:
from imblearn.under_sampling import RandomUnderSampler
undersample= RandomUnderSampler(random_state=42)
X_unsampled, Y_unsampled= undersample.fit_resample(X_train, Y_train)

In [43]:
rf_model= RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_unsampled, Y_unsampled)

#Evaluation
y_pred= rf_model.predict(X_val)

print("Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))

Accuracy: 0.6717724288840262

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.68      0.74       309
           1       0.49      0.65      0.56       148

    accuracy                           0.67       457
   macro avg       0.65      0.67      0.65       457
weighted avg       0.70      0.67      0.68       457



# Humour Dataset

In [90]:
import pandas as pd

#Load
file_path = "/content/train.csv.1"

df = pd.read_csv(file_path)

df.head()


,Sentence,Tag
0,"Jyotiraditya Scindia is like ""Rassi jal gayee ...",1
1,Ishant Sharma ko bahut late utaara.,1
2,.@twinitisha neeche plug nikla hua hai..,0
3,Aaj agar India final me hota to kam se kam New...,0
4,3 stages of life of Mechanical Engineer:\n\n1)...,1


In [46]:
missing_values = df.isnull().sum()
print("\nMissing Values per Column:")
print(missing_values)


Missing Values per Column:
Sentence    0
Tag         0
dtype: int64


In [47]:

sentences = df["Sentence"].tolist()
tags = df["Tag"].tolist()

#Display first few sentences
for i in range(5):
    print(f"Sentence: {sentences[i]}")
    print(f"Tag: {tags[i]}")
    print("-" * 50)

Sentence: Jyotiraditya Scindia is like "Rassi jal gayee aunty ki bal nahi gaya"pic.twitter.com/RP1sQpr90l
Tag: 1
--------------------------------------------------
Sentence: Ishant Sharma ko bahut late utaara.
Tag: 1
--------------------------------------------------
Sentence: .@twinitisha neeche plug nikla hua hai..
Tag: 0
--------------------------------------------------
Sentence: Aaj agar India final me hota to kam se kam New Zealand se to accha hi khelta.
Tag: 0
--------------------------------------------------
Sentence: 3 stages of life of Mechanical Engineer:

1) Janam 

2) "Abe wo dekh IT wali ko.. Kya maal hai behen ki lodi"

3) Maut
Tag: 1
--------------------------------------------------


## Words level N grams

In [48]:
vectorizer= CountVectorizer(ngram_range=(1,3))  #Preprocessing
X_train= vectorizer.fit_transform(sentences)
Y_train= tags

#Display first few features
print("Sample Features (First 10):")
print(vectorizer.get_feature_names_out()[:10])


Sample Features (First 10):
['05' '05 pm' '05 pm sbi' '08wk3fbarn' '0cbqklrlti' '0esduufve3'
 '0mtywt9pex' '0mywhis5qb' '0rv8cuju7g' '0uyfld97nf']


### Multinomial Naive Bayes

In [49]:
model= MultinomialNB()
model.fit(X_train, Y_train)

MultinomialNB()

In [50]:
df_val= pd.read_csv('/content/val.csv.1')

df_val.head()

,Sentence,Tag
0,.@OfficeOfRG #HappyBdayPM wala hashtag use kar...,0
1,"""Main kuch bhi dekh sakta hun bas teri ankhon ...",1
2,No rainbow pic cause waise bhi rangeeley kism ...,0
3,"Hamein aur jeene ki khwahish na hoti, agar rum...",1
4,Ladkiyo ke parts pe comment karke gaali khaane...,1


In [51]:
#Extracting sentences from validtion set
val_sentences= df_val["Sentence"].tolist()
val_labels= df_val["Tag"].tolist()


### Char level N-Grams

In [52]:

vectorizer1= CountVectorizer(analyzer='char', ngram_range=(1,4))

X_all= vectorizer1.fit_transform(sentences + val_sentences)
X_train= vectorizer1.transform(sentences)
X_val= vectorizer1.transform(val_sentences)

model.fit(X_train, Y_train)

#Evaluation
y_pred = model.predict(X_val)

print("Model Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))


Model Accuracy: 0.6644067796610169

Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.39      0.49       119
           1       0.67      0.85      0.75       176

    accuracy                           0.66       295
   macro avg       0.65      0.62      0.62       295
weighted avg       0.66      0.66      0.64       295



### To make similar number of counts

# Best model (multinomial-nb) givng the F1 score of 77%

In [53]:
from imblearn.over_sampling import RandomOverSampler

oversample= RandomOverSampler(random_state=42)

X_resampled, Y_resampled= oversample.fit_resample(X_train, Y_train)

In [54]:
pd.Series(Y_resampled).value_counts()

,count
1,1407
0,1407


In [55]:
model.fit(X_resampled, Y_resampled)

y_pred = model.predict(X_val)

print("Model Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))


Model Accuracy: 0.6949152542372882

Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.48      0.56       119
           1       0.70      0.84      0.77       176

    accuracy                           0.69       295
   macro avg       0.69      0.66      0.66       295
weighted avg       0.69      0.69      0.68       295



### SVM

### Improving accuracy by neglecting stop words etc

In [56]:
import re

#Preprocessing
def preprocess_text(text):
    text= text.lower()  #lowercase
    text= re.sub(r'\d+', '', text)  #Remove numbers
    text= re.sub(r'[^\w\s]', '', text)  #Remove punctuation
    return text


sentences2= [preprocess_text(sentence) for sentence in sentences]
val_sentences2= [preprocess_text(sentence) for sentence in val_sentences]

vectorizer_new= CountVectorizer(ngram_range=(1,3), stop_words='english')  #Removes common words
X_train1= vectorizer_new.fit_transform(sentences2)
X_val1= vectorizer_new.transform(val_sentences2)


In [57]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
svm_model= SVC(kernel='linear', class_weight='balanced')

In [58]:
svm_model.fit(X_train1, Y_train)

#Evaluation
y_pred= svm_model.predict(X_val1)

print("Model Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))

Model Accuracy: 0.6949152542372882

Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.68      0.64       119
           1       0.77      0.70      0.73       176

    accuracy                           0.69       295
   macro avg       0.69      0.69      0.69       295
weighted avg       0.70      0.69      0.70       295



## Logistic Regression

In [59]:
logistic_model= LogisticRegression(max_iter=500)
logistic_model.fit(X_resampled, Y_resampled)

#Evaluation
y_pred= logistic_model.predict(X_val)

print("Logistic Regression Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))

Logistic Regression Accuracy: 0.6847457627118644

Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.59      0.60       119
           1       0.73      0.75      0.74       176

    accuracy                           0.68       295
   macro avg       0.67      0.67      0.67       295
weighted avg       0.68      0.68      0.68       295



## Random Forest

In [60]:
from sklearn.ensemble import RandomForestClassifier
rf_model= RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_resampled, Y_resampled)

RandomForestClassifier(random_state=42)

In [61]:
y_pred= rf_model.predict(X_val)
print("Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))

Accuracy: 0.6949152542372882

Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.61      0.62       119
           1       0.74      0.75      0.75       176

    accuracy                           0.69       295
   macro avg       0.68      0.68      0.68       295
weighted avg       0.69      0.69      0.69       295



## UnderSampling

In [62]:
from imblearn.under_sampling import RandomUnderSampler
undersample= RandomUnderSampler(random_state=42)
X_unsampled, Y_unsampled= undersample.fit_resample(X_train, Y_train)

In [63]:
rf_model= RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_unsampled, Y_unsampled)

#Evaluation
y_pred= rf_model.predict(X_val)

print("Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))

Accuracy: 0.6983050847457627

Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.72      0.66       119
           1       0.78      0.68      0.73       176

    accuracy                           0.70       295
   macro avg       0.69      0.70      0.69       295
weighted avg       0.71      0.70      0.70       295



# Sarcasm Dataset

In [70]:
import pandas as pd

#Load Dataset
file_path= "/content/train.csv.2"


df= pd.read_csv(file_path)

df.head()


,Sentence,Tag
0,"MashaAllah, jo log meri Black display peh chor...",1
1,"Sanjeev, Suchitra Sen, and Gulzar. #SanjeevKu...",0
2,Politics to harami hai par tu sabse bada haram...,0
3,aaj cricket khiladi neelam honge... khair khil...,0
4,#PAKvWXI Hamay cricket fever mai Etna mast Na...,0


In [71]:
missing_values = df.isnull().sum()
print("\nMissing Values per Column:")
print(missing_values)


Missing Values per Column:
Sentence    0
Tag         0
dtype: int64


In [72]:

sentences = df["Sentence"].tolist()
tags = df["Tag"].tolist()

#Display first few sentences
for i in range(5):
    print(f"Sentence: {sentences[i]}")
    print(f"Tag: {tags[i]}")
    print("-" * 50)

Sentence: MashaAllah, jo log meri Black display peh chorh k chalay gaye thy, wo wapis arahay ab?  #Irony #FollowUnfollow
Tag: 1
--------------------------------------------------
Sentence: Sanjeev, Suchitra Sen, and Gulzar.  #SanjeevKumar #Gulzar #GulzarSaab #suchitra #HappyBirthday #Legend #Friends #Bollywood pic.twitter.com/yN4uPkl53A
Tag: 0
--------------------------------------------------
Sentence: Politics to harami hai par tu sabse bada haramzaada hai .modi se anne se pehle tu desh bhakt tha ab tu jihadi hai.laanat hai .
Tag: 0
--------------------------------------------------
Sentence: aaj cricket khiladi neelam honge... khair khiladi to khel ko hi neelam krne lage gain...fir khud neelam hona kaun so badi baat hair.
Tag: 0
--------------------------------------------------
Sentence: #PAKvWXI  Hamay cricket fever mai Etna mast Nahi hona chahiay k mazloom #rohingya ki nasal kashi bhool jaye.."meri masti mai shaoor hai "
Tag: 0
--------------------------------------------------


## Words level N grams

In [73]:
vectorizer= CountVectorizer(ngram_range=(1,3))  #Preprocessing
X_train= vectorizer.fit_transform(sentences)
Y_train= tags

#Display first few features
print("Sample Features (First 10):")
print(vectorizer.get_feature_names_out()[:10])


Sample Features (First 10):
['000' '000 feeling' '000 feeling shame' '01' '01 september'
 '01 september 2006' '03' '03 hamare' '03 hamare yaha' '03 ke']


### Multinomial Naive Bayes

In [74]:
model= MultinomialNB()
model.fit(X_train, Y_train)

MultinomialNB()

In [75]:
df_val= pd.read_csv('/content/val.csv.2')

df_val.head()

,Sentence,Tag
0,Aek tarf pakistan ke dushman pakistan ke khila...,0
1,Lalu ne bihar ki seva karte hue kitne ghotaale...,0
2,Politics me har admi besharam ho jata hai.... ...,0
3,Kudakudhinge dhuvasthamee? #Maldives #Politics,0
4,Kya ram-rahim naam ke gunde ke bhakto ko manma...,0


In [76]:
#Extracting sentences from validtion set
val_sentences= df_val["Sentence"].tolist()
val_labels= df_val["Tag"].tolist()


### Char level N-Grams

In [77]:

vectorizer1= CountVectorizer(analyzer='char', ngram_range=(1,4))

X_all= vectorizer1.fit_transform(sentences + val_sentences)
X_train= vectorizer1.transform(sentences)
X_val= vectorizer1.transform(val_sentences)

model.fit(X_train, Y_train)

#Evaluation
y_pred = model.predict(X_val)

print("Model Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))


Model Accuracy: 0.9180952380952381

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.96      0.95       474
           1       0.59      0.51      0.55        51

    accuracy                           0.92       525
   macro avg       0.77      0.74      0.75       525
weighted avg       0.91      0.92      0.92       525



### To make similar number of counts

In [78]:
from imblearn.over_sampling import RandomOverSampler

oversample= RandomOverSampler(random_state=42)

X_resampled, Y_resampled= oversample.fit_resample(X_train, Y_train)

In [79]:
pd.Series(Y_resampled).value_counts()

,count
1,3797
0,3797


In [80]:
model.fit(X_resampled, Y_resampled)

y_pred = model.predict(X_val)

print("Model Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))


Model Accuracy: 0.9561904761904761

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.96      0.98       474
           1       0.72      0.90      0.80        51

    accuracy                           0.96       525
   macro avg       0.85      0.93      0.89       525
weighted avg       0.96      0.96      0.96       525



### SVM

### Improving accuracy by neglecting stop words etc

In [81]:
import re

#Preprocessing
def preprocess_text(text):
    text= text.lower()  #lowercase
    text= re.sub(r'\d+', '', text)  #Remove numbers
    text= re.sub(r'[^\w\s]', '', text)  #Remove punctuation
    return text


sentences2= [preprocess_text(sentence) for sentence in sentences]
val_sentences2= [preprocess_text(sentence) for sentence in val_sentences]

vectorizer_new= CountVectorizer(ngram_range=(1,3), stop_words='english')  #Removes common words
X_train1= vectorizer_new.fit_transform(sentences2)
X_val1= vectorizer_new.transform(val_sentences2)


In [82]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
svm_model= SVC(kernel='linear', class_weight='balanced')

In [83]:
svm_model.fit(X_train1, Y_train)

#Evaluation
y_pred= svm_model.predict(X_val1)

print("Model Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))

Model Accuracy: 0.9638095238095238

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       474
           1       0.86      0.75      0.80        51

    accuracy                           0.96       525
   macro avg       0.92      0.87      0.89       525
weighted avg       0.96      0.96      0.96       525



## Logistic Regression

# Best model which is giving the F1 score as 85%

In [84]:
logistic_model= LogisticRegression(max_iter=500)
logistic_model.fit(X_resampled, Y_resampled)

#Evaluation
y_pred= logistic_model.predict(X_val)

print("Logistic Regression Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))

Logistic Regression Accuracy: 0.9714285714285714

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.98       474
           1       0.85      0.86      0.85        51

    accuracy                           0.97       525
   macro avg       0.92      0.92      0.92       525
weighted avg       0.97      0.97      0.97       525



## Random Forest

In [85]:
from sklearn.ensemble import RandomForestClassifier
rf_model= RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_resampled, Y_resampled)

RandomForestClassifier(random_state=42)

In [86]:
y_pred= rf_model.predict(X_val)
print("Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))

Accuracy: 0.9561904761904761

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.96      0.98       474
           1       0.72      0.90      0.80        51

    accuracy                           0.96       525
   macro avg       0.85      0.93      0.89       525
weighted avg       0.96      0.96      0.96       525



## UnderSampling

In [87]:
from imblearn.under_sampling import RandomUnderSampler
undersample= RandomUnderSampler(random_state=42)
X_unsampled, Y_unsampled= undersample.fit_resample(X_train, Y_train)

In [88]:
rf_model= RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_unsampled, Y_unsampled)

#Evaluation
y_pred= rf_model.predict(X_val)

print("Accuracy:", accuracy_score(val_labels, y_pred))
print("\nClassification Report:\n", classification_report(val_labels, y_pred))

Accuracy: 0.96

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.96      0.98       474
           1       0.72      0.96      0.82        51

    accuracy                           0.96       525
   macro avg       0.86      0.96      0.90       525
weighted avg       0.97      0.96      0.96       525



# Question 2  (Sequence Labelling Task)

In [ ]:
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/train_data.txt
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/test_data.txt
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/noisy_test_data.txt

--2025-03-18 15:45:31--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/train_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375849 (367K) [text/plain]
Saving to: ‘train_data.txt’

train_data.txt      100%[===================>] 367.04K  --.-KB/s    in 0.004s  

2025-03-18 15:45:31 (85.8 MB/s) - ‘train_data.txt’ saved [375849/375849]

--2025-03-18 15:45:31--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/test_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP reques

In [ ]:
def load_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            sentence = []
            for token in line.strip().split():
                word, tag = token.rsplit('/', 1)  # Split word and tag
                sentence.append((word, tag))
            data.append(sentence)
    return data

# Load train and test data from files
train_data_file = '/content/train_data.txt'  # Path to your training data file
test_data_file = '/content/test_data.txt'    # Path to your test data file
noisy_test_data_file = '/content/noisy_test_data.txt'  # Path to your noisy test data file

train_data = load_data(train_data_file)
test_data = load_data(test_data_file)
noisy_test_data = load_data(noisy_test_data_file)

# Print a sample from the training data
print(train_data[0])

[('He', 'PRON'), ('let', 'VERB'), ('her', 'PRON'), ('tell', 'VERB'), ('him', 'PRON'), ('all', 'PRT'), ('about', 'ADP'), ('the', 'DET'), ('church', 'NOUN'), ('.', '.')]


In [ ]:
#Read test data
with open(test_data_file, "r", encoding="utf-8") as f:
    test= f.readlines()

#Read noisy test data
with open(noisy_test_data_file, "r", encoding="utf-8") as f:
    noisy_test= f.readlines()


In [ ]:
from collections import defaultdict

#Transition probabilities P(tag2 | tag1)
transition= defaultdict(lambda: defaultdict(int))
tag_c= defaultdict(int)

#Emission probabilities P(word | tag)
emission= defaultdict(lambda: defaultdict(int))
word_c= defaultdict(int)

#Count occurrences in training data
for sentence in train_data:
    prev_tag= "<START>"
    for word, tag in sentence:
        transition[prev_tag][tag]+= 1
        emission[tag][word]+= 1
        tag_c[tag]+= 1
        word_c[word]+= 1
        prev_tag= tag

#Convert counts to probabilities
transition_probs= {tag1: {tag2: count / sum(transition[tag1].values()) for tag2, count in transition[tag1].items()}
    for tag1 in transition}

emission_probs= {tag: {word: count / tag_c[tag] for word, count in emission[tag].items()}
    for tag in emission}

print("Sample Transition Probabilities:", list(transition_probs.items())[:3])
print("\nSample Emission Probabilities:", list(emission_probs.items())[:3])

Sample Transition Probabilities: [('<START>', {'PRON': 0.149, 'NOUN': 0.1505, 'ADV': 0.0965, 'DET': 0.201, 'ADP': 0.1345, '.': 0.083, 'PRT': 0.031, 'VERB': 0.0485, 'ADJ': 0.0365, 'NUM': 0.016, 'CONJ': 0.053, 'X': 0.0005}), ('PRON', {'VERB': 0.7080118694362018, 'PRT': 0.018991097922848664, 'ADV': 0.05222551928783383, '.': 0.10801186943620178, 'CONJ': 0.016023738872403562, 'DET': 0.01661721068249258, 'ADJ': 0.00830860534124629, 'ADP': 0.05400593471810089, 'NOUN': 0.008902077151335312, 'PRON': 0.00830860534124629, 'NUM': 0.0005934718100890207}), ('VERB', {'PRON': 0.05244415668501133, 'PRT': 0.05843314988669472, 'NOUN': 0.09954677889284558, 'DET': 0.1633214632567174, 'VERB': 0.18420200712204596, 'ADP': 0.17691809647134996, 'ADV': 0.10488831337002266, 'ADJ': 0.059080608611201034, '.': 0.07672385885399806, 'CONJ': 0.016672062156037552, 'NUM': 0.0076076400129491744, 'X': 0.0001618646811265782})]

Sample Emission Probabilities: [('PRON', {'He': 0.06112759643916914, 'her': 0.024925816023738872,

## Viterbi Algorithm

In [ ]:
import numpy as np

def viterbi(sentence, transition_probs, emission_probs, tags):
    sentence= sentence.split()
    n= len(sentence)
    numtags= len(tags)

    #Initialize DP table and backpointers
    dp= np.zeros((numtags, n))
    backpointer= np.zeros((numtags, n), dtype=int)

    for i, tag in enumerate(tags):
        dp[i][0]= transition_probs.get("<START>", {}).get(tag, 1e-6) * emission_probs.get(tag, {}).get(sentence[0], 1e-6)

    #Fill DP table
    for t in range(1, n):
        for j, tag in enumerate(tags):
            max_prob, max_index= 0, 0
            for i, prev_tag in enumerate(tags):
                prob= dp[i][t-1] * transition_probs.get(prev_tag, {}).get(tag, 1e-6) * emission_probs.get(tag, {}).get(sentence[t], 1e-6)
                if prob> max_prob:
                    max_prob, max_index= prob, i
            dp[j][t]= max_prob
            backpointer[j][t]= max_index

    #Backtracking
    best_seq= []
    best_index= np.argmax(dp[:, n-1])
    for t in range(n-1, -1, -1):
        best_seq.append(tags[best_index])
        best_index= backpointer[best_index][t]

    return list(reversed(best_seq))  #Reverse the sequence to get correct order

#Extract all unique POS tags from training data
all_tags = list(tag_c.keys())


## Evaluation on Test and Noisy Test data

In [ ]:
from sklearn.metrics import accuracy_score

#Function to evaluate the Viterbi Algorithm on test data
def evaluate(test_lines, transition_probs, emission_probs, tags):
    correct_tags= []
    predicted_tags= []

    for line in test_lines:
        tokens= line.strip().split()
        words, true_tags= zip(*[token.rsplit("/", 1) for token in tokens])
        predicted= viterbi(" ".join(words), transition_probs, emission_probs, tags)

        correct_tags.extend(true_tags)
        predicted_tags.extend(predicted)

    #Accuracy
    return accuracy_score(correct_tags, predicted_tags)

In [ ]:
#Evaluate model on clean test data
test_accuracy= evaluate(test, transition, emission, all_tags)

#Evaluate model on noisy test data
noisy_test_accuracy= evaluate(noisy_test, transition, emission, all_tags)

#Results
print(f"Accuracy on Clean Test Data: {test_accuracy*100:.2f}%")
print(f"Accuracy on Noisy Test Data: {noisy_test_accuracy*100:.2f}")

<ipython-input-5-14ab98af9ff6>:20: RuntimeWarning: overflow encountered in scalar multiply
  prob= dp[i][t-1] * transition_probs.get(prev_tag, {}).get(tag, 1e-6) * emission_probs.get(tag, {}).get(sentence[t], 1e-6)
<ipython-input-5-14ab98af9ff6>:20: RuntimeWarning: overflow encountered in scalar multiply
  prob= dp[i][t-1] * transition_probs.get(prev_tag, {}).get(tag, 1e-6) * emission_probs.get(tag, {}).get(sentence[t], 1e-6)


Accuracy on Clean Test Data: 88.68%
Accuracy on Noisy Test Data: 80.52
